In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/kaggle/input/arabic-news-dataset/processed_data.csv')

# Display basic information about the dataset
print(df.info())
print(df.head())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8399 entries, 0 to 8398
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     8399 non-null   object
 1   Content   8386 non-null   object
 2   Link      8399 non-null   object
 3   Category  8399 non-null   object
dtypes: object(4)
memory usage: 262.6+ KB
None
                                               Title  \
0  تيرتل تري لابز للتقنيات الحيويه تحصل علي تمويل...   
1     تعد دراسه الجدوي لمشروعك الناشء 6 عوامل اساسيه   
2  منح جاءزه نوبل الطب لاكتشاف مجال التهاب الكبد ...   
3  هيءه الغذاء والدواء الامريكيه لقاح مودرنا حقق ...   
4  حظر بروتين الجهاز المناعي يساعد علي محاربه عدو...   

                                             Content  \
0  جاءت جوله التمويل التمهيديه الاولي للشركه بقيا...   
1  تعد دراسه الجدوي متطلبا اساسيا لنجاح اي مشروع ...   
2  وقالت لجنه نوبل للمره الاولي التاريخ يمكن الان...   
3  نشرت الوكاله وثاءق علي الانترنت اعدها موظفوها ...   
4 

In [2]:
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize

# Ensure necessary resources are downloaded
nltk.download('punkt')

# Function to get the most common words in each category
def get_common_words(df, category_col, text_col, num_words=10):
    common_words = {}
    categories = df[category_col].unique()

    for category in categories:
        category_text = ' '.join(df[df[category_col] == category][text_col].dropna().tolist())
        words = word_tokenize(category_text.lower())
        common_words[category] = Counter(words).most_common(num_words)

    return common_words

# Get the most common words for each category
common_words_per_category = get_common_words(df, 'Category', 'Content')
common_words_per_category


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'ريادة أعمال': [('علي', 16899),
  ('الي', 10824),
  ('ان', 9679),
  ('الاعمال', 7623),
  ('خلال', 5327),
  ('الشركات', 4465),
  ('شركه', 3749),
  ('الناشءه', 3633),
  ('و', 3006),
  ('او', 2911)],
 'علوم وتكنولوجيا': [('ان', 12000),
  ('علي', 11549),
  ('الي', 8507),
  ('او', 2595),
  ('يمكن', 2125),
  ('انه', 1729),
  ('عام', 1470),
  ('خلال', 1444),
  ('اكثر', 1367),
  ('بشكل', 1343)],
 'أخرى': [('علي', 23877),
  ('ان', 21817),
  ('الي', 21705),
  ('المتحده', 8620),
  ('الامم', 6384),
  ('العام', 5370),
  ('او', 4124),
  ('جميع', 3501),
  ('خلال', 3392),
  ('الامين', 3389)]}

In [3]:
import re
from collections import defaultdict

# Function to tokenize text
def simple_tokenize(text):
    text = text.lower()
    tokens = re.findall(r'\b\w+\b', text)
    return tokens

# Function to get the most common words in each category
def get_common_words_simple(df, category_col, text_col, num_words=10):
    common_words = defaultdict(Counter)
    categories = df[category_col].unique()

    for category in categories:
        category_text = df[df[category_col] == category][text_col].dropna().tolist()
        for text in category_text:
            tokens = simple_tokenize(text)
            common_words[category].update(tokens)

    # Return the most common words for each category
    return {category: common_words[category].most_common(num_words) for category in categories}

# Get the most common words for each category using simple tokenization
common_words_per_category_simple = get_common_words_simple(df, 'Category', 'Content')
common_words_per_category_simple


{'ريادة أعمال': [('علي', 16908),
  ('الي', 10834),
  ('ان', 9682),
  ('الاعمال', 7625),
  ('خلال', 5328),
  ('الشركات', 4465),
  ('شركه', 3749),
  ('الناشءه', 3633),
  ('و', 3020),
  ('او', 2915)],
 'علوم وتكنولوجيا': [('ان', 12000),
  ('علي', 11551),
  ('الي', 8509),
  ('او', 2596),
  ('يمكن', 2126),
  ('انه', 1729),
  ('عام', 1470),
  ('خلال', 1445),
  ('اكثر', 1368),
  ('بشكل', 1344)],
 'أخرى': [('علي', 23881),
  ('ان', 21817),
  ('الي', 21710),
  ('المتحده', 8620),
  ('الامم', 6384),
  ('العام', 5373),
  ('او', 4124),
  ('جميع', 3502),
  ('خلال', 3393),
  ('الامين', 3389)]}

In [4]:
# Define the rule-based classification function
def classify_category(text):
    if pd.isna(text):
        return 'Unknown'
    
    text = text.lower()
    if any(word in text for word in ["الاعمال", "الشركات", "شركه", "الناشءه"]):
        return 'ريادة أعمال'
    elif any(word in text for word in ["يمكن", "عام", "بشكل"]):
        return 'علوم وتكنولوجيا'
    elif any(word in text for word in ["المتحده", "الامم", "الامين"]):
        return 'أخرى'
    else:
        return 'Unknown'

# Apply the function to classify categories based on content
df['Predicted_Category'] = df['Content'].apply(classify_category)

# Display the distribution of predicted categories
predicted_category_distribution = df['Predicted_Category'].value_counts()
predicted_category_distribution


Predicted_Category
علوم وتكنولوجيا    4106
ريادة أعمال        3737
Unknown             445
أخرى                111
Name: count, dtype: int64